In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_initial_data(years=None):
    url = "https://www.gt-world-challenge-europe.com/results?filter_season_id=0&filter_meeting_id=0&filter_race_id="
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Извлекаем доступные сезоны
    seasons_options = soup.select('#filter_season_id option')
    seasons = {option.text.strip(): option['value'] for option in seasons_options if option['value']}

    # Фильтруем сезоны по указанным годам, если таковые имеются
    if years:
        seasons = {year: seasons[year] for year in years if year in seasons}
    
    # Извлекаем доступные встречи
    meetings_options = soup.select('#filter_meeting_id option')
    meetings = {option.text.strip(): option['value'] for option in meetings_options if option['value']}
    
    return seasons, meetings


def parse_years(input_years):
    if isinstance(input_years, list):
        return [str(year) for year in input_years]
    elif isinstance(input_years, str) and '-' in input_years:
        start_year, end_year = map(int, input_years.split('-'))
        return [str(year) for year in range(start_year, end_year - 1, -1)]
    elif input_years:
        return [str(input_years)]
    else:
        return None


def get_race_data(input_years=None):
    base_url = "https://www.gt-world-challenge-europe.com/results"
    years = parse_years(input_years)
    seasons, meetings = fetch_initial_data(years)
    results = []

    for season_name, season_id in seasons.items():
        for meeting_name, meeting_id in meetings.items():
            url = f"{base_url}?filter_season_id={season_id}&filter_meeting_id={meeting_id}&filter_race_id="
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            race_options = soup.select('#filter_race_id option')

            races = []
            for option in race_options:
                if option['value']: # Исключаем пустые значения
                    races.append({"race_id": option['value'], "name": option.text.strip()})
            
            # Добавляем информацию о сезоне, встрече и гонках в результат
            results.append({
                "season": season_name,
                "season_id": season_id,
                "meeting": meeting_name,
                "meeting_id": meeting_id,
                "races": races
            })

    return results


def load_race_data(input_years=None):
    # Преобразуем ввод в список годов, если это необходимо
    years = parse_years(input_years)  # Используем функцию parse_years для обработки ввода
    all_data = get_race_data(years)  # Получаем данные, используя уже существующую логику фильтрации
    return all_data



def fetch_race_results(filter_season, filter_meeting, filter_race, race_data):
    # Ищем соответствующие ID для сезона, встречи и гонки
    season_id, meeting_id, race_id = None, None, None
    for data in race_data:
        if data['season'] == filter_season:
            season_id = data['season_id']
            if data['meeting'] == filter_meeting:
                meeting_id = data['meeting_id']
                for race in data['races']:
                    if race['name'] == filter_race:
                        race_id = race['race_id']
                        break
                break
    
    if not season_id or not meeting_id or not race_id:
        raise ValueError("Один из параметров не найден. Проверьте правильность введенных данных.")
    
    # Формирование запроса с найденными ID и получение данных гонки
    final_url = f"https://www.gt-world-challenge-europe.com/results?filter_season_id={season_id}&filter_meeting_id={meeting_id}&filter_race_id={race_id}"
    response = requests.get(final_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Парсинг данных гонки
    rows = soup.select('.table__body tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        if not cols:  # Если ряд полностью пустой, пропускаем его
            continue
        row_data = [col.text.strip() for col in cols]
        while len(row_data) < 9:  # Дополняем ряд до ожидаемого количества столбцов
            row_data.append(None)
        data.append(row_data)
    
    # Создание DataFrame
    df_columns = ['Pos', 'Car #', 'Class', 'Drivers', 'Team', 'Car', 'Time', 'Laps', 'Gap']
    df = pd.DataFrame(data, columns=df_columns if data else [])
    
    return df


In [31]:
my_race_data = load_race_data(2023)

In [33]:
df = fetch_race_results("2023", "Barcelona", "Main Race", my_race_data)

df.head()

,Pos,Car #,Class,Drivers,Team,Car,Time,Laps,Gap
0,1,51,Pro Cup,"Alessio Rovera, Robert Shwartzman, Nicklas Nie...",AF Corse - Francorchamps Motors,Ferrari 296 GT3,1:41.540,88,
1,2,71,Pro Cup,"Antonio Fuoco, Daniel Serra, Davide Rigon",AF Corse - Francorchamps Motors,Ferrari 296 GT3,1:41.665,88,0.422
2,3,96,Pro Cup,"Thomas Preining, Laurin Heinrich, Dennis Olsen",Rutronik Racing,Porsche 911 GT3 R (992),1:41.562,88,2.328
3,4,777,Pro Cup,"Maro Engel, Luca Stolz, Fabian Schiller",Mercedes-AMG Team AlManar,Mercedes-AMG GT3 EVO,1:41.467,88,2.770
4,5,88,Pro Cup,"Raffaele Marciello, Jules Gounon, Timur Bogusl...",AKKODIS ASP Team,Mercedes-AMG GT3 EVO,1:42.162,88,4.413


In [32]:
df1 = fetch_race_results("2023", "Barcelona", "Qualifying 1", my_race_data)

df1.head()

,Pos,Car #,Class,Drivers,Team,Car,Time,Laps,Gap
0,1,51,Pro Cup,"Alessio Rovera, Robert Shwartzman, Nicklas Nie...",AF Corse - Francorchamps Motors,Ferrari 296 GT3,1:39.297,,
1,2,71,Pro Cup,"Antonio Fuoco, Daniel Serra, Davide Rigon",AF Corse - Francorchamps Motors,Ferrari 296 GT3,1:39.320,,0.023
2,3,63,Pro Cup,"Mirko Bortolotti, Andrea Caldarelli, Jordan Pe...",Iron Lynx,Lamborghini Huracan GT3 EVO 2,1:39.414,,0.117
3,4,98,Pro Cup,"Philipp Eng, Marco Wittmann, Nick Yelloly",Rowe Racing,BMW M4 GT3,1:39.896,,0.599
4,5,54,Pro Cup,"Ayhancan Güven, Sven Müller, Christian Engelhart",Dinamic GT,Porsche 911 GT3 R (992),1:39.928,,0.631
